In [1]:
def baric_corr(carpeta):
    from astropy.io import fits
    from astropy.time import Time
    from astropy.coordinates import SkyCoord, EarthLocation
    import astropy.units as u
    from astropy.constants import c
    import numpy as np
    import glob 
    import os
    
    patron = os.path.join(carpeta, '*modificado.fits')#Gran*-star*_sum_m*_fsr1.05_AIR_modificado.fits
    archivos= glob.glob(patron)

    if not archivos:
        print(f"no se encontraron archivos en: {carpeta}")
        return
    print(f'se encontraron {len(archivos)} archivos')
    v_s= []
    for archivo in archivos:
        print(f'procesando {archivo}')
        # leer el fits
        hdul = fits.open(archivo)
        hdr = hdul[0].header
        flux = hdul[0].data

        ra = hdr['RA']
        dec = hdr['DEC']
        date = hdr['DATE-OBS']
        obj = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))
        location = EarthLocation.of_site("La Silla Observatory")
        location = location.to_geodetic()
        t = Time(date, format = 'mjd', location=location)
        barycorr = obj.radial_velocity_correction(obstime=t)
        v_bc = barycorr.to(u.km/u.s).value
        v_s.append(v_bc)
    return(np.median(v_s))   
    

In [2]:
st1 = baric_corr('/home/nacho/molecfit_test/mis_outputs_star1/')
st3 = baric_corr('/home/nacho/molecfit_test/mis_outputs_star3/')
st4 = baric_corr('/home/nacho/molecfit_test/mis_outputs_star4/')
st7 = baric_corr('/home/nacho/molecfit_test/mis_outputs_star7/')
st8 = baric_corr('/home/nacho/molecfit_test/mis_outputs_star8/')
st9 = baric_corr('/home/nacho/molecfit_test/mis_outputs_star9/')

se encontraron 13 archivos
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m43_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m44_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m45_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m46_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m47_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m48_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m51_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m52_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m53_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/

# para pasar de vacio a aire

In [1]:
def vac_to_air(wavelength_vac):
    import numpy as np
    wl_vac = np.atleast_1d(wavelength_vac)
    
    sigma2 = (1e4 / wl_vac)**2
    # indice de refracción del aire (n-1) * 10^8
    n_minus_1 = 6432.8 + 2949810.0 / (146.0 - sigma2) + 25540.0 / (41.0 - sigma2)
    n_minus_1 *= 1e-8
    
    n = 1.0 + n_minus_1
    
    # Conversión de vacío a aire
    wl_air = wl_vac / n
    
    print(f"el indice de refraccion es:{n} ")
    return wl_air
def vacuum_a_air(ruta_carpeta):
    from astropy.io import fits
    from shutil import copyfile
    import os 
    import glob
    
    # Buscar todos los archivos que terminen en par_tac.fits en la carpeta
    patron = os.path.join(ruta_carpeta, '*par_tac.fits')
    archivos = glob.glob(patron)
    
    if not archivos:
        print(f"No se encontraron archivos que terminen en 'par_tac.fits' en la carpeta: {ruta_carpeta}")
        return
    
    print(f"Se encontraron {len(archivos)} archivos para procesar:")
    for archivo in archivos:
        print(f"  - {os.path.basename(archivo)}")
    
    # Procesar cada archivo
    for archivo in archivos:
        print(f"\nProcesando: {archivo}")
        
        # crear la copia
        # 1. Copia de seguridad
        backup = archivo.replace('.fits', '_backup_vacuum.fits')
        if not os.path.exists(backup):
            copyfile(archivo, backup)
            print(f"  Copia de seguridad creada: {backup}")
        else:
            print(f"  Copia de seguridad ya existe: {backup}")
        0
        # Procesar el archivo FITS
        with fits.open(backup) as hdul:
            datos = hdul[1].data
            long_onda = datos['LAMBDA']
            

            # sobreescribir los datos
            datos['LAMBDA'] = vac_to_air(long_onda)

            hdul.flush()
        
        print(f"  ¡Listo! {backup} ahora tiene longitudes de onda en AIRE")
        print(f"  Diferencia típica ~1.7–2.7 Å (más corto que en vacío)")
    
    print(f"\nProcesamiento completado. Total de archivos procesados: {len(archivos)}")

In [2]:
archivos = "/home/nacho/molecfit_test/mis_outputs_star1//"
vacuum_a_air(archivos)

Se encontraron 13 archivos para procesar:
  - Gran01-star3_sum_m44_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m45_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m46_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m47_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m48_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m51_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m52_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m53_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m54_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m55_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m56_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m57_fsr1.05_AIR_modificado.par_tac.fits
  - Gran01-star3_sum_m43_fsr1.05_AIR_modificado.par_tac.fits

Procesando: /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m44_fsr1.05_AIR_modificado.par_tac.fits
  Copia de seguridad ya existe: /home/nacho/molecfi

# aplicar CCF

In [3]:
def cross_corr_apli(carpeta):
    """esta funcion se encarga de hacer el CCF y aplicar también la correccion por velocidad baricentrica
    input: 
    carpeta: corresponde a la carpeta en donde se encuentran nuestros archivos fits, ya sean los archivos
    fits de calctrans o los archivos fits que necesita la funcon baric_corr
    
    outputs: entrega un .dat con el CCF ademas de graficos png para observar el fit"""
    import glob
    import os
    import re
    import subprocess

    # carpeta 
    grillas = "/home/nacho/molecfit_test/winered_R28k_airGrid/"

    # buscar los ordenes
    archivos = sorted(glob.glob(os.path.join(carpeta, "**", "*.par_tac_backup_vacuum.fits"), recursive=True)) #par_tac_backup_vacuum / par_tac.fits /AIR.fits
    
    if not archivos:
        print(" No se encontraron archivos _par_tac.fits.")
        return
    ordenes_validos = []
    for ruta in archivos:
        match = re.search(r'_m(\d+)_', ruta)
        if match:
            orden = int(match.group(1))
            ordenes_validos.append((orden, ruta))
    # ordenar por el numero de orden 
    ordenes_validos.sort(key=lambda x: x[0])

    for orden, ruta in ordenes_validos:
        print(f"\n Procesando orden {orden}...")
        cmd = [
            "python3", "rvcorr_fitsTableSpec.py",
            ruta,
            grillas,
            "--vmin", "0",
            "--vmax", "200",
            "-n"
        ]
        try:
            subprocess.run(cmd, check=True)
            print(f"Orden {orden} procesado con éxito.")
        except subprocess.CalledProcessError:
            print(f" Error procesando orden {orden}.")

In [4]:
#estrella 1 
carpeta = "/home/nacho/molecfit_test/mis_outputs_star1//"
#carpeta = "/home/nacho/molecfit_test/mis_outputsVAC_star1/"
cross_corr_apli(carpeta)


 Procesando orden 43...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  195.50
[Info] Best template chisquare:  0.0017178314496686714
[Info] ccf Vobs:  195.575
[Info] Final Vobs:  195.584
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m43_fsr1.05_AIR_modificado.par_tac_backup_vacuum_vcorr.fits
[Info] Results written in  /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m43_fsr1.05_AIR_modificado.par_tac_backup_vacuum_RV.dat

Orden 43 procesado con éxito.

 Procesando orden 44...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  195.00
[Info] Best template chisquare:  0.0015872406302690635
[Info] ccf Vobs:  195.325
[Info] Final Vobs:  195.321
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m44_fsr1.05_AIR_modificado.

In [3]:
#estrella 3 
#carpeta = "/home/nacho/molecfit_test/mis_outputs_star3/"
carpeta = "/home/nacho/molecfit_test/ordenes_star3/"
#vacuum_a_air(carpeta)
cross_corr_apli(carpeta)


 Procesando orden 43...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  116.00
[Info] Best template chisquare:  0.007994945597909088
[Info] ccf Vobs:  116.050
[Info] Final Vobs:  116.052
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/ordenes_star3/Gran01-star3_sum_m43_fsr1.05_AIR_vcorr.fits
[Info] Results written in  /home/nacho/molecfit_test/ordenes_star3/Gran01-star3_sum_m43_fsr1.05_AIR_RV.dat

Orden 43 procesado con éxito.

 Procesando orden 44...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  116.00
[Info] Best template chisquare:  0.009205254025119965
[Info] ccf Vobs:  115.825
[Info] Final Vobs:  115.829
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/ordenes_star3/Gran01-star3_sum_m44_fsr1.05_AIR_vcorr.fits
[Info] Results written in  /home/nacho/molecfit_test/ordenes_star3/Gran01-star3_

In [4]:
#estrella 4
carpeta = "/home/nacho/molecfit_test/ordenes_star4/"
#carpeta = "/home/nacho/molecfit_test/mis_outputs_star4/"
#vacuum_a_air(carpeta)
cross_corr_apli(carpeta)


 Procesando orden 43...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  104.00
[Info] Best template chisquare:  0.00664276633392107
[Info] ccf Vobs:  103.900
[Info] Final Vobs:  103.909
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/ordenes_star4/Gran1-star4_sum_m43_fsr1.05_AIR_vcorr.fits
[Info] Results written in  /home/nacho/molecfit_test/ordenes_star4/Gran1-star4_sum_m43_fsr1.05_AIR_RV.dat

Orden 43 procesado con éxito.

 Procesando orden 44...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  104.50
[Info] Best template chisquare:  0.011767880766032706
[Info] ccf Vobs:  105.875
[Info] Final Vobs:  105.874
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/ordenes_star4/Gran1-star4_sum_m44_fsr1.05_AIR_vcorr.fits
[Info] Results written in  /home/nacho/molecfit_test/ordenes_star4/Gran1-star4_sum_m

In [5]:
#estrella 7
carpeta = "/home/nacho/molecfit_test/ordenes_star7/"
#carpeta = "/home/nacho/molecfit_test/mis_outputs_star7/"
#vacuum_a_air(carpeta)
cross_corr_apli(carpeta)


 Procesando orden 43...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  105.00
[Info] Best template chisquare:  0.004316307291615352
[Info] ccf Vobs:  105.050
[Info] Final Vobs:  105.057
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/ordenes_star7/Gran1-star7_sum_m43_fsr1.05_AIR_vcorr.fits
[Info] Results written in  /home/nacho/molecfit_test/ordenes_star7/Gran1-star7_sum_m43_fsr1.05_AIR_RV.dat

Orden 43 procesado con éxito.

 Procesando orden 44...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  105.50
[Info] Best template chisquare:  0.007400280246527305
[Info] ccf Vobs:  107.125
[Info] Final Vobs:  107.126
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/ordenes_star7/Gran1-star7_sum_m44_fsr1.05_AIR_vcorr.fits
[Info] Results written in  /home/nacho/molecfit_test/ordenes_star7/Gran1-star7_sum_

In [6]:
#estrella 8
carpeta = "/home/nacho/molecfit_test/ordenes_star8/"
#carpeta = "/home/nacho/molecfit_test/mis_outputs_star8/"
#vacuum_a_air(carpeta)
cross_corr_apli(carpeta)


 Procesando orden 43...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  116.50
[Info] Best template chisquare:  0.007999633071164474
[Info] ccf Vobs:  117.125
[Info] Final Vobs:  116.903
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/ordenes_star8/Gran01-star8_sum_m43_fsr1.05_AIR_vcorr.fits
[Info] Results written in  /home/nacho/molecfit_test/ordenes_star8/Gran01-star8_sum_m43_fsr1.05_AIR_RV.dat

Orden 43 procesado con éxito.

 Procesando orden 44...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  116.00
[Info] Best template chisquare:  0.008675902113701305
[Info] ccf Vobs:  117.000
[Info] Final Vobs:  117.003
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/ordenes_star8/Gran01-star8_sum_m44_fsr1.05_AIR_vcorr.fits
[Info] Results written in  /home/nacho/molecfit_test/ordenes_star8/Gran01-star8_

In [7]:
#estrella 9
carpeta = "/home/nacho/molecfit_test/ordenes_star9/"
#carpeta = "/home/nacho/molecfit_test/mis_outputs_star9/"
#vacuum_a_air(carpeta)
cross_corr_apli(carpeta)


 Procesando orden 43...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  102.50
[Info] Best template chisquare:  0.006225346249074193
[Info] ccf Vobs:  102.550
[Info] Final Vobs:  102.557
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/ordenes_star9/Gran1-star9_sum_m43_fsr1.05_AIR_vcorr.fits
[Info] Results written in  /home/nacho/molecfit_test/ordenes_star9/Gran1-star9_sum_m43_fsr1.05_AIR_RV.dat

Orden 43 procesado con éxito.

 Procesando orden 44...
[Info] Number of templates:  215
[Info] Inputs allow to compute velocity shifts in(    0.00,  200.00)
[Info] Preliminary Vobs:  103.50
[Info] Best template chisquare:  0.009551824112534624
[Info] ccf Vobs:  104.700
[Info] Final Vobs:  104.696
[Info] Corrected spectrum written in:  /home/nacho/molecfit_test/ordenes_star9/Gran1-star9_sum_m44_fsr1.05_AIR_vcorr.fits
[Info] Results written in  /home/nacho/molecfit_test/ordenes_star9/Gran1-star9_sum_

# ahora vamos a buscar las velocidades radiales en los archivos 


In [10]:
import os

# Carpetas donde se encuentran los outputs de rvcorr_fitsTableSpec
# carpetas = [
#     "/home/nacho/molecfit_test/mis_outputs_star1/",
#     "/home/nacho/molecfit_test/mis_outputs_star3/",
#     "/home/nacho/molecfit_test/mis_outputs_star4/",
#     "/home/nacho/molecfit_test/mis_outputs_star7/",
#     "/home/nacho/molecfit_test/mis_outputs_star8/",
#     "/home/nacho/molecfit_test/mis_outputs_star9/"
# ]
carpetas = [
    "/home/nacho/molecfit_test/ordenes_star1/",
    "/home/nacho/molecfit_test/ordenes_star3/",
    "/home/nacho/molecfit_test/ordenes_star4/",
    "/home/nacho/molecfit_test/ordenes_star7/",
    "/home/nacho/molecfit_test/ordenes_star8/",
    "/home/nacho/molecfit_test/ordenes_star9/"
]
for carpeta in carpetas:
    print("\n Revisando:", carpeta)
    for root, dirs, files in os.walk(carpeta):
        for f in files:
            if f.lower().endswith(".dat"):
                print("   ", f)
def extraer_rvs(carpeta):
    import os
    import glob
    import numpy as np

    # Buscar archivos .dat dentro de la carpeta
    archivos = sorted(glob.glob(os.path.join(carpeta, "**", "*AIR_RV.dat"), recursive=True))# par_tac_RV.dat

    if not archivos:
        print(f" No se encontraron archivos .dat en {carpeta}")
        return []

    rvs = []
    for ruta in archivos:
        with open(ruta, "r") as f:
            for linea in f:
                linea = linea.strip()
                if not linea or linea.startswith("#"):
                    continue
                partes = linea.split()
                if len(partes) > 1:
                    try:
                        rv = float(partes[1])  # segundo dato numérico
                        rvs.append(rv)
                    except ValueError:
                        continue  # ignora líneas no numéricas
    print(f"{os.path.basename(os.path.normpath(carpeta))}: {len(rvs)} valores leidos ")
    print(f"{os.path.basename(os.path.normpath(carpeta))}: la deviacion estandar es: {np.std(rvs)}")
    return rvs

# Procesar todas las carpetas
rvs = []

for c in carpetas:
    rvs.append(extraer_rvs(c))




 Revisando: /home/nacho/molecfit_test/ordenes_star1/
   ➜ Gran01-star3_sum_m43_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m44_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m45_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m46_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m47_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m48_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m51_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m52_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m53_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m54_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m55_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m56_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m57_fsr1.05_AIR_RV.dat

 Revisando: /home/nacho/molecfit_test/ordenes_star3/
   ➜ Gran01-star3_sum_m43_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m44_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m45_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m46_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m47_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m48_fsr1.05_AIR_RV.dat
   ➜ Gran01-star3_sum_m51_fsr1.05_AIR

In [20]:
import numpy as np
car1 = "/home/nacho/molecfit_test/mis_outputs_star1/"
car3 = "/home/nacho/molecfit_test/mis_outputs_star3/"
car4 = "/home/nacho/molecfit_test/mis_outputs_star4/"
car7 = "/home/nacho/molecfit_test/mis_outputs_star7/"
car8 = "/home/nacho/molecfit_test/mis_outputs_star8/"
car9 = "/home/nacho/molecfit_test/mis_outputs_star9/"
# car1 = "/home/nacho/molecfit_test/ordenes_star1/"
# car3 = "/home/nacho/molecfit_test/ordenes_star3/"
# car4 = "/home/nacho/molecfit_test/ordenes_star4/"
# car7 = "/home/nacho/molecfit_test/ordenes_star7/"
# car8 = "/home/nacho/molecfit_test/ordenes_star8/"
# car9 = "/home/nacho/molecfit_test/ordenes_star1/"
rbcs=[baric_corr(car1),baric_corr(car3),baric_corr(car4),baric_corr(car7),baric_corr(car8),baric_corr(car9)]

rvs_final = []
xd = 0
while xd < 6:
    print(f"velocidad radial estrella {xd} = {np.median(rvs[xd])+rbcs[xd]} km/s")
    rvs_final.append(np.median(rvs[xd])+rbcs[xd])
    xd += 1


se encontraron 13 archivos
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m43_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m44_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m45_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m46_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m47_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m48_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m51_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m52_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/mis_outputs_star1/Gran01-star3_sum_m53_fsr1.05_AIR_modificado.fits
procesando /home/nacho/molecfit_test/

In [ ]:
# rvs = [113,116,104,105,116,103]
# rvs_ff = []
# a = 0
# while a<6:
#     rvs_ff.append(rvs[a]+rvs_final[a])
#     a +=1
# rvs_ff
